<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/Seq2Seq_sample_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt


In [3]:
!pip install Seq2Seq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for Seq2Seq: filename=seq2seq-0.0.4-py2.py3-none-any.whl size=2031 sha256=c69466368ef1d0a0a12d6b3ecce62e9c793321ab2be0b83cbdce076c138a7c8d
  Stored in directory: /root/.cache/pip/wheels/2d/e3/d6/6a773e18c55ec3c577f6ad52d28448806632bd859079f401d8
Successfully built Seq2Seq


In [ ]:
vocab_size = 256
x_ = list(map(ord, 'hello'))
y_ = list(map(ord, 'hola'))
x = torch.LongTensor(x_)
y = torch.LongTensor(y_)

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, vocab_size, hidden_size):
      super(Seq2Seq, self).__init__()
      self.n_layers = 1
      self.hidden_size = hidden_size
      self.embedding = nn.Embedding(vocab_size, hidden_size)
      self.encoder = nn.GRU(hidden_size, hidden_size)
      self.decoder = nn.GRU(hidden_size, hidden_size)
      self.project = nn.Linear(hidden_size, vocab_size)

  def forward(self, inputs, targets):
      initial_state = self.__init__state()
      embedding = self.embedding(inputs).unsqueeze(1)
      encoder_output, encoder_state = self.encoder(embedding, initial_state)
      decoder_state = encoder_state
      decoder_input = torch.LongTensor([0])

      outputs = []
      for i in range(targets.size()[0]):
          decoder_input = self.embedding(decoder_input).unsqueeze(1)
          decoder_output, decoder_state = self.decoder(decoder_input, decoder_state)

          projection = self.project(decoder_output)
          outputs.append(projection)

          decoder_input = torch.LongTensor([targets[i]])
      outputs = torch.stack(outputs).squeeze()
      return outputs

      def _init_state(self, batch_size =1):
          weight = next(self.parameters()).data
          return weight.new(self.n_layers, batch_size, self.hidden_size).zero_() 

In [ ]:
seq2seq = Seq2Seq(vocab_size, 16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=1e-3)

In [ ]:
log = []
for i in range(1000):
    prediction = seq2seq(x,y)
    loss = criterion(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_val = loss.data
    log.append(loss_val)
    if i % 100 == 0:
       print('\n 반복 : %d 오차 : %s' % (i, loss_val.item()))
       _, top1 = prediction.data.topk(1,1)
       print([chr(c) for c in top1.squeeze().numpy().tolist()])

plt.plot(log)
plt.ylabel('cross entropy loss')
plt.show()